# BrickByte - DataDog Example

This notebook demonstrates how to sync monitoring and analytics data from DataDog to Databricks using BrickByte.

## Prerequisites
- DataDog account
- API Key (found at https://app.datadoghq.com/organization-settings/api-keys)
- Application Key (found at https://app.datadoghq.com/organization-settings/application-keys)
- Databricks workspace with Unity Catalog

## Note
Ensure your API and Application keys have the appropriate permissions for the data you want to sync.


In [ ]:
%run ./_setup


In [ ]:
from brickbyte import BrickByte

bb = BrickByte(
    sources=["source-datadog"],
    destination="destination-databricks",
    destination_install="git+https://github.com/park-peter/brickbyte.git#subdirectory=integrations/destination-databricks-py"
)
bb.setup()


In [ ]:
import airbyte as ab

FORCE_FULL_REFRESH = True
cache = bb.get_or_create_cache()

# Configure the DataDog source
# Documentation: https://docs.airbyte.com/integrations/sources/datadog
source = ab.get_source(
    "source-datadog",
    config={
        "api_key": "",            # DataDog API Key
        "application_key": "",    # DataDog Application Key
        "site": "datadoghq.com",  # Options: datadoghq.com, us3.datadoghq.com, datadoghq.eu, etc.
        "start_date": "2024-01-01T00:00:00Z",
        "end_date": "2024-12-31T23:59:59Z",
    },
    local_executable=bb.get_source_exec_path("source-datadog")
)
source.check()
source.select_all_streams()
print("Available streams:", source.get_available_streams())


In [ ]:
# Configure the Databricks destination
destination = ab.get_destination(
    "destination-databricks",
    config={
        "server_hostname": "",  # e.g., "adb-xxx.azuredatabricks.net"
        "http_path": "",        # e.g., "/sql/1.0/warehouses/abc123"
        "token": "",            # Your Databricks PAT
        "catalog": "",          # Unity Catalog name
        "schema": "",           # Target schema
    },
    local_executable=bb.get_destination_exec_path()
)

write_result = destination.write(source, cache=cache, force_full_refresh=FORCE_FULL_REFRESH)
print("Sync completed!")


In [ ]:
bb.cleanup()


In [ ]:
# Cleanup virtual environments
bb.cleanup()


## Query Your Data

After the sync completes, you can query your DataDog data:

```sql
-- View monitors and their status
SELECT 
    _airbyte_emitted_at,
    _airbyte_data:id AS monitor_id,
    _airbyte_data:name AS name,
    _airbyte_data:type AS type,
    _airbyte_data:overall_state AS status
FROM your_catalog.your_schema._airbyte_raw_monitors
ORDER BY _airbyte_data:name
LIMIT 20;

-- View recent incidents
SELECT 
    _airbyte_data:id AS incident_id,
    _airbyte_data:attributes.title AS title,
    _airbyte_data:attributes.severity AS severity,
    _airbyte_data:attributes.state AS state,
    _airbyte_data:attributes.created AS created_at
FROM your_catalog.your_schema._airbyte_raw_incidents
ORDER BY _airbyte_data:attributes.created DESC
LIMIT 20;

-- View dashboards
SELECT 
    _airbyte_data:id AS dashboard_id,
    _airbyte_data:title AS title,
    _airbyte_data:author_handle AS author,
    _airbyte_data:created_at AS created_at
FROM your_catalog.your_schema._airbyte_raw_dashboards
LIMIT 20;
```
